# Instalacion de Dependencias



In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 33.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1a0cb6e981bf6c7a1aa1857e2425190b232f7051e07e791e90f3d6c5c1976d00
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697

# Creacion del Spark Context

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Primer Cuatrimestre de 2018. Examen parcial, tercera oportunidad

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un archivo con el siguiente formato: (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. 

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a)  Las categorías que hayan incrementado su tasa de resolución en al menos un 10%  en el último trimestre, con respecto al trimestre anterior.
b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos) 

In [ ]:
# (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).
cases = [("2019-01-01", 1, "case 1", 2, "otro delito", -1, -1), 
         ("2019-06-01", 2, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 3, "case 2", 3, "robo", -1, -1),         
         ("2019-06-01", 4, "case 2", 1, "robo", -1, -1),         
         ("2019-06-01", 5, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 6, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 9, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 10, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 60, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 70, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 80, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 90, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 100, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 600, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 700, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 800, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 900, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 1000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 6000, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7000, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8000, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 9000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 10000, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 92, "case 2", 2, "hurto", -1, -1),
         ("2019-06-01", 93, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 94, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 95, "case 2", 3, "hurto", -1, -1),
         ("2019-09-01", 96, "case 2", 2, "hurto", -1, -1),
 
        ]

# (id_caso, respuesta)
batsignal = [(1,1),
         (2,1),
         (3,0),
         (4,0),
         (5,1),
         (6,0),
         (7,1),
         (8,0),
         (9,1),
         (100,1),
         (600,0),
         (700,1),
         (800,0),
         (900,1),
         (1000,1),
         (6000,0),
         (7000,1),
         (8000,0),
         (9000,1),
         (10000,1),
         (92,0),
         (93,0),             
         (94,0),
         (95,0),             
         (96,1)             
        ]

In [ ]:
casesRdd = sc.parallelize(cases)
batsignalRdd = sc.parallelize(batsignal)

In [ ]:
casesFiltered = casesRdd.filter(lambda x: x[0] > '2019-04-00' and x[0] < '2019-10-01' and x[3] != 1)

In [ ]:
casesByCat = casesFiltered.map(lambda x: ((x[4],1 if x[0] > '2019-07-00' else 0), (1 if x[3]==2 else 0, 1)))

In [ ]:
casesByCat.collect()

[(('robo', 0), (1, 1)),
 (('robo', 0), (0, 1)),
 (('robo', 0), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (1, 1)),
 (('robo', 1), (0, 1)),
 (('robo', 1), (1, 1)),
 (('hurto', 0), (1, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 0), (0, 1)),
 (('hurto', 1), (1, 1))]

In [ ]:
casesByCatSum = casesByCat.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))

In [ ]:
casesByCatSum.collect()

[(('robo', 1), (10, 15)),
 (('hurto', 0), (1, 4)),
 (('robo', 0), (2, 3)),
 (('hurto', 1), (1, 1))]

In [ ]:
casesByCatSum.map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().map(lambda x:(x[0],list(x[1]))).collect()

[('hurto', [(0, (1, 4)), (1, (1, 1))]), ('robo', [(1, (10, 15)), (0, (2, 3))])]

In [ ]:
def aumento(l):
  if l[0][0] == 0:
    return l[1][1][0] / l[1][1][1] - l[0][1][0] / l[0][1][1] > 0.1
  else:
    return  l[0][1][0] / l[0][1][1] - l[1][1][0] / l[1][1][1] > 0.1

In [ ]:
casesByCatSum.map(lambda x: (x[0][0],(x[0][1],x[1]))).groupByKey().map(lambda x:(x[0],list(x[1])))\
.filter(lambda x: aumento(x[1])).map(lambda x: x[0]).collect()

['hurto']

Punto b

In [ ]:
casesPropiedad = casesRdd.filter(lambda x: x[4] in ['robo','hurto'])

In [ ]:
casesPropiedad.collect()

[('2019-06-01', 2, 'case 2', 2, 'robo', -1, -1),
 ('2019-06-01', 3, 'case 2', 3, 'robo', -1, -1),
 ('2019-06-01', 4, 'case 2', 1, 'robo', -1, -1),
 ('2019-06-01', 5, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 6, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 7, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 8, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 9, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 10, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 60, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 70, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 80, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 90, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 100, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 600, 'case 2', 2, 'robo', -1, -1),
 ('2019-09-01', 700, 'case 2', 3, 'robo', -1, -1),
 ('2019-09-01', 800, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 900, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 1000, 'case 2', 1, 'robo', -1, -1),
 ('2019-09-01', 6000, 'case 2', 2, 'robo', -1, -1),

In [ ]:
casesJoinSignal = casesPropiedad.map(lambda x: (x[1],x[4])).leftOuterJoin(batsignalRdd)

In [ ]:
casesJoinSignal.collect()

[(4, ('robo', 0)),
 (8, ('robo', 0)),
 (60, ('robo', None)),
 (80, ('robo', None)),
 (100, ('robo', 1)),
 (600, ('robo', 0)),
 (700, ('robo', 1)),
 (800, ('robo', 0)),
 (900, ('robo', 1)),
 (1000, ('robo', 1)),
 (6000, ('robo', 0)),
 (7000, ('robo', 1)),
 (8000, ('robo', 0)),
 (9000, ('robo', 1)),
 (10000, ('robo', 1)),
 (92, ('hurto', 0)),
 (96, ('hurto', 1)),
 (5, ('robo', 1)),
 (9, ('robo', 1)),
 (93, ('hurto', 0)),
 (2, ('robo', 1)),
 (6, ('robo', 0)),
 (10, ('robo', None)),
 (70, ('robo', None)),
 (90, ('robo', None)),
 (94, ('hurto', 0)),
 (3, ('robo', 0)),
 (7, ('robo', 1)),
 (95, ('hurto', 0))]

In [ ]:
casesJoinSignal.map(lambda x: (x[1][0],(1, 1 if x[1][1] == 1 else 0)))\
.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1])).map(lambda x: (x[0],float(x[1][1])/x[1][0])).collect()

[('hurto', 0.2), ('robo', 0.4583333333333333)]